In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import pandas as pd
dtrain = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
dval = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
dtest = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')

In [11]:
dtrain

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [7]:
dtest

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


# FirstApproach: 

**Consider this example as binary classification with only toxic**

In [13]:
dtrain = dtrain[['id','comment_text','toxic']]
dtrain.head(5)

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [15]:
each_len = []

for k in dtrain['comment_text']:
    each_len.append(len(k.split(' ')))
    
dtrain['comment_len'] = each_len

In [128]:
dtrain_sorted = dtrain.groupby('comment_len')[['comment_text']].max().sort_values(by='comment_len', ascending=False).reset_index()

dtrain_sorted

,comment_len,comment_text
0,2834,""" \n = = R e g a r d i n g K a r e l = = ..."
1,2681,""" \n\n ==Hey sexy== \n Quit sucking so much ha..."
2,2321,C O M M I E - F U C K E R C O M M I E - F U C ...
3,2273,"""\n\nAs near as I can tell, wikipedia has no a..."
4,2071,== me editing bad. == \n\n well' i want to be ...
...,...,...
1020,5,일이삼사오육칠팔구하고십이요 에헤헤 으헤 으헤 으허허
1021,4,괴로운 변명을 하는 사람이다-_-;;
1022,3,→ English Wikipedia←
1023,2,ขอบคุณครับคำแนะนำครับ ใส่แหล่งอ้างอิงเรียบร้อย...


In [129]:
print(dtrain_sorted.iloc[0]['comment_text'])
print(dtrain_sorted.iloc[0]['comment_len'])

"   
 = = R e g a r d i n g   K a r e l = =  
 
  J o y ,  
 
  I ' m   a   b i t   c o n c e r n e d   o v e r   y o u r   e d i t s   t o   [ [ C h a r l e s   K a r e l   B o u l e y ] ]   o v e r   t h e   p a s t   f e w   m o n t h s ,   a s   y o u r   e d i t s   c o m e   a c r o s s   a s   f a n   e d i t s .     A   n u m b e r   o f   m o r e   c o n t r o v e r s i a l   c o m m e n t s   h a v e   b e e n   r e m o v e d   f r o m   t h e   a r t i c l e   ( g e t t i n g   f i r e d   f r o m   K F I ,   g e t t i n g   i n   t r o u b l e   f o r   h i s   R e a g a n   c o m m e n t s ,   e t c . ) ,   l e a v i n g   t h e   a r t i c l e   c o m e   a c r o s s   w i t h   a   c o m p l e t e l y   f a v o r a b l e   v i e w p o i n t .  
 
  Q u e s t i o n      D o   y o u   k n o w   K a r e l   i n   p e r s o n ?     A r e   y o u   a   f r i e n d   o f   h i s ?     A r e   y o u   a   f a n ?     A   f r e q u e n t   u s e r   o f   h i s   c h a t   r o o

In [139]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

def roc_auc_cal(pred,y):
    a=roc_auc_score(y, pred)
    return a

In [138]:
#EXAMPLE
'''
# Simulated dataset
y_true = [0, 0, 1, 1]
y_scores = [0.1, 0.4, 0.35, 0.8]

print(roc_auc_cal(y_scores,y_true))  ---- 0.75
'''

'\n# Simulated dataset\ny_true = [0, 0, 1, 1]\ny_scores = [0.1, 0.4, 0.35, 0.8]\n\nprint(roc_auc_cal(y_scores,y_true))  ---- 0.75\n'

In [140]:
dtrain

,id,comment_text,toxic,comment_len
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,42
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,18
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,42
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,112
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,13
...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,108
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,15
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,15
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,27


In [148]:
from sklearn.model_selection import train_test_split
y = dtrain['toxic']
x = dtrain['comment_text']
x_train,x_test,y_train,y_test = train_test_split(x.values,y.values,random_state=42,test_size=0.25, shuffle=True)


In [153]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping


token = text.Tokenizer(num_words=None)

ImportError: cannot import name 'text' from 'keras.preprocessing' (/usr/local/lib/python3.10/dist-packages/keras/api/preprocessing/__init__.py)